# Morpheus Murano App 1.0.0

This Murano App uses a microk8s cluster to provide a space to test NVIDIA Morpheus on the Nectar cloud. To seperate Morpheus from any other deployments you may wish to test, all the `morpheus` namespace has been used. 

This app includes a jupyter lab server to make things more user friendly during experimentation and NginX to make accessing it and the MLFlow UI easier.

The morpheus stack includes:

- [MLFlow](https://mlflow.org/) to manage models
- [Triton](https://developer.nvidia.com/nvidia-triton-inference-server) as an inference server
- [Kafka](https://kafka.apache.org/) as a data broker

You can check the health of the cluster using:
`microk8s kubectl -n morpheus get all`

You can access the Jupyter Lab at `{address}/`
You can access the MLFlow UI at `{address}/mlflow`

In [ ]:
!microk8s kubectl -n morpheus get all

For more information on usage, have a read of the [quick start guide](https://docs.nvidia.com/morpheus/morpheus_quickstart_guide.html) which this notebook is based on.

NVIDIA recommends that you only run one pipeline at once, this means that usually you would need to uninstall any active pipelines before replacing it during testing. By default this app is running the the `Morpheus SDK Client` in 'sleep mode' under the `helper` release name. Rather than destroying and recreating the sdk, we will be interfacing with it through the CLI directly, building pipelines with the CLI.


# Using Morpheus

Morpheus can be run in two ways:

- Run off file
  
    NVIDIA provides several example models and datasets to test morpheus with. You can browse these on their [GitHub](https://github.com/nv-morpheus/Morpheus/tree/branch-22.06/models). The quick start guide includes several examples which involve loading in this data into the input topics and storing outputs in a file. You can also upload your own data to the container using Jupyter's interface.

- Run off kafka topic
  
    NVIDIA Morpheus can interface with the kafka data broker, pulling new data from an input topic and pushing inferences to an output topic. NVIDIA provides scripts to load data from a file into the input topic to simulate a data stream, but in production this should be gathered from the system you are monitoring.


This implementation of Morpheus is deployed using four containers:

- The *ai-engine* container, which runs NVIDIA Triton, listening for HTTP requests on port 8000, gRPC on port 8001 and metrics in the prometheus format on port 8002.
- The *sdk-cli-helper* container, which runs the SDK and the Jupyter Lab (this container). We are using NginX to manage the connections to Jupyter to make your life easier.
- The *broker* container, which runs Kafka, listening on port 9092. It is also exposed on 30092 on the main machine allowing you to feed it data, though it will be blocked by default by Nectar's security. You will need to enable this security rule if you'd like to do this, ensure that you are not working with sensitive data if you'd like to experiment with this.
- The zookeeper
- The *mlflow* container, which runs MLFlow, listening on port 5000. It's UI is also exposed on 30500 on the main machine allowing you to view the models you have deployed.

In [ ]:
# Let's set up some aliases to execute commands in the containers to make our lives easier
x_sdk = "microk8s kubectl -n morpheus exec -it sdk-cli-helper -- "
x_morpheus = "microk8s kubectl -n morpheus exec -it sdk-cli-helper -- /opt/conda/envs/morpheus/bin/morpheus"
x_broker = "microk8s kubectl -n morpheus exec deploy/broker -c broker -- "
x_mlflow_python = "microk8s kubectl -n morpheus exec -it deploy/mlflow -- /opt/conda/envs/mlflow/bin/python"
x_mlflow = "microk8s kubectl -n morpheus exec -it deploy/mlflow -- /opt/conda/envs/mlflow/bin/mlflow"

In [ ]:
!$x_sdk curl -v ai-engine:8000/v2/health/ready

In [ ]:
!$x_morpheus

In [ ]:
!$x_broker kafka-topics.sh

In [ ]:
!$x_mlflow

# Sensitive Information Detection (SID)

NVIDIA has provided us with a set of example models and datasets to get started in the `/models` and `/models/datasets` directory of the SDK container. In this example we will look at the SID model.

To share these files with MLFlow, copy it to the `/common` directory which is mapped to `/opt/morpheus/common` on the host. 

In [ ]:
!$x_sdk cp -RL /workspace/models /common

## MLFlow

The MLFlow service is used for managing and deploying models. We can use NVIDIA's scripts to deploy models to the Triton ai-engine.

In [ ]:
!$x_mlflow_python publish_model_to_mlflow.py \
      --model_name sid-minibert-onnx \
      --model_directory /common/models/triton-model-repo/sid-minibert-onnx \
      --flavor triton 

In [ ]:
!$x_mlflow deployments create -t triton \
      --flavor triton \
      --name sid-minibert-onnx \
      -m models:/sid-minibert-onnx/1 \
      -C "version=1"

## Kafka

To run our pipeline we will need to create kafka topics for input and output

In [ ]:
# Create an input topic
!$x_broker kafka-topics.sh\
      --create \
      --bootstrap-server broker:9092 \
      --replication-factor 1 \
      --partitions 3 \
      --topic input

In [ ]:
# Create an output topic
!$x_broker kafka-topics.sh\
      --create \
      --bootstrap-server broker:9092 \
      --replication-factor 1 \
      --partitions 3 \
      --topic output

In [ ]:
# View the topics we've just created
!$x_broker --list  --zookeeper zookeeper:2181

In [ ]:
!$x_sdk ls examples/data/

## Pipeline examples
Now that we've finished setting up, we can set up some pipelines! Morpheus constructs pipelines made up of 'stages', including preprocessing and postprocessing steps accellerated with NVIDIA RAPIDS, which will allow you to build something to handle your data stream in near real time. If Morpheus's provided pipeline stages do not fit your needs, it also allows you to extend its capabilities with a [custom stage written in Python or C++](https://docs.nvidia.com/morpheus/developer_guide/guides/1_simple_python_stage.html#background).

Morpheus provides scripts to simulate an input stream from a file or by streaming a file into the data broker, but in a production setting you would feed your data into the pipeline using the input topic we set up earlier.

For more examples of these pipelines have a read through these [example workflows](https://docs.nvidia.com/morpheus/morpheus_quickstart_guide.html#example-workflows).

In [ ]:
# Simulate a datastream using the pcap_dump dataset file
!$x_morpheus --log_level=DEBUG run \
      --num_threads=3 \
      --edge_buffer_size=4 \
      --use_cpp=True \
      --pipeline_batch_size=1024 \
      --model_max_batch_size=32 \
      pipeline-nlp \
        --model_seq_length=256 \
        from-file --filename=./examples/data/pcap_dump.jsonlines \
        monitor --description 'FromFile Rate' --smoothing=0.001 \
        deserialize \
        preprocess --vocab_hash_file=data/bert-base-uncased-hash.txt --truncation=True --do_lower_case=True --add_special_tokens=False \
        monitor --description='Preprocessing rate' \
        inf-triton --force_convert_inputs=True --model_name=sid-minibert-onnx --server_url=ai-engine:8001 \
        monitor --description='Inference rate' --smoothing=0.001 --unit inf \
        add-class \
        serialize --exclude '^ts_' \
        to-file --filename=/common/data/sid-minibert-onnx-output.jsonlines --overwrite

# Next Steps
Using NVIDIA's pretrained models and examples are a good place to start to get a grasp of the framework, but fine tuning is necessary for any model deployed in a production setting.

NVIDIA has provided training scripts and notebooks for each of their models for you to play around with to retrain their models. If their models do not adapt well to your use case you can also look at [hugging face's model repository](https://huggingface.co/models) as a place to start and train your own model.

### WARNING
Note that many of these scripts and notebooks were moved from their original repositories when being compiled into the Morpheus container. This means that any paths present within these scripts and notebooks **will not accurately reflect where they are in the container**. NVIDIA also **has not provided any environment files** to help us recreate the environments used to train these models. The datasets will likely be somewhere in the folder we copied to `/opt/morpheus/common/` earlier. It may be easier to browse the file structure on the [NVIDIA Morpheus GitHub](https://github.com/nv-morpheus/Morpheus/tree/branch-22.09/models). It will be up to you track down the datasets and recreate the environments if you wish to use the tools that NVIDIA have provided as a starting ground. Alternatively you can look through these resources to get inspiration on how you can train your own models to make use of this framework. 

Good luck!

In [ ]:
# Copy NVIDIA's training scripts into our root directory so that we can easily access their scripts and notebooks
!cp /opt/morpheus/common/models/training-tuning-scripts/ ./ -r